# HIRE ME please <33333333

Nvm, here is the code. Enjoy!

## Imports...

In [8]:
import requests
from pathlib import Path
import os
from tqdm.notebook import tqdm
import pandas as pd
import time

## Getting all county ID:s

In [9]:
url = "https://stiftelser.lansstyrelsen.se/Search/GetCarouselItems"
response = requests.get(url)

all_county_ids = []

if response.ok:
    response_dict = response.json()
else:
    print("Error:", response.status_code, response.text)

for county in response_dict['items']:
    path = Path(county['imagesUrl'])
    county_name = os.path.splitext(path.name)[0]
  
    all_county_ids.append({
        "name": county_name,
        "searchLink":county['searchLink']}
        )
    
all_county_ids

[{'name': 'norrbotten', 'searchLink': '25'},
 {'name': 'orebro', 'searchLink': '18'},
 {'name': 'ostergotland', 'searchLink': '05'},
 {'name': 'skane', 'searchLink': '12'},
 {'name': 'stockholm', 'searchLink': '01'},
 {'name': 'vastra-gotaland', 'searchLink': '14'}]

## Getting all foundations for each county, and then getting all members from each foundation... 

In [21]:
# Adding some delays to avoid being kicked out... Always a good idea. 
delay_between_each_req = 0.1 # Wait 0.1 second between each req. Good practice. 
delay_between_county = 180 # 3 minutes before getting from each county
delay_every_500_req = 10 # Add 10 sec delay every 1000 reqeust

nbr_req = 1

# Creating a DataFrame with following columns
df = pd.DataFrame(columns=["county", "foundation_name", "org_nbr", "person_name", "role"])

# Search word used for filtering on role
search_word = "styrelse"

# Looping over all counties... 
for county in all_county_ids:

  url = "https://stiftelser.lansstyrelsen.se/search/avdsearch" 
  data = {"regmyndighet": county['searchLink']}

  response = requests.post(url, json=data)

  if response.ok:
    response_dict = response.json()
    print("Scraping",county['name'])
  else:
      print("Error:", response.status_code, response.text)

  # all foundation in county...
  all_foundations = response_dict['stiftelser']

  # Looping over all foundations...
  for foundation in tqdm(all_foundations, desc="Fetching data"):

    # Get board members for each foundation
    url = "https://stiftelser.lansstyrelsen.se/search/fetchstift"
    data = {"fetchedByStiftelseID": foundation['id']}

    # Pause out of respect
    if nbr_req % 500 == 0:  
      time.sleep(delay_every_500_req) 
    else:
      time.sleep(delay_between_each_req)  

    response = requests.post(url, json=data)
    nbr_req += 1

    if response.ok:
      response_dict = response.json()
    else:
        print("Error:", response.status_code, response.text)

    board_members = response_dict['roller']

    filtered_board_members = [p for p in board_members if search_word in p['typ'].lower()]

    # Looping over all members and appending a new row for each board member...
    for person in filtered_board_members:
      df.loc[len(df)] = [
        county['name'],
        foundation['namn'],
        foundation['orgnrNoMinus'][:-4] + "-" + foundation['orgnrNoMinus'][-4:],
        person['namn'],
        person['typ']
      ]
    
  # Quick brake... 
  time.sleep(delay_between_county)


    

Scraping norrbotten


Fetching data:   0%|          | 0/1029 [00:00<?, ?it/s]

Scraping orebro


Fetching data:   0%|          | 0/3340 [00:00<?, ?it/s]

Scraping ostergotland


...

Fetching data:   0%|          | 0/3080 [00:00<?, ?it/s]

Scraping stockholm


Fetching data:   0%|          | 0/6156 [00:00<?, ?it/s]

Scraping vastra-gotaland


Fetching data:   0%|          | 0/2683 [00:00<?, ?it/s]

## Saving to csv...

In [22]:
# 3. Export to CSV
df.to_csv("board_members.csv", index=False)